In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
#with h5py.File("data_water/eri_water.hdf5", "r") as f:
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_10/integral.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_10/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

10
(10, 27)
(27, 27)
rl errV: 0.00023694902148543046
abs errV: 0.0007546522262828301
errt: 0.0008451096565123393
errh: 0.00013317184525868722
errht: 0.00024513110698447764


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
# #load Hamiltonian generated by above coefficients
#H_correct = scipy.io.mmread('/.mtx').tocsr()
# e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
# e_ground = e1

# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)

Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [4]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [5]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

e_ground = -12.407196480366359

[1, 16, 70, 108, 162, 232, 162, 108, 70, 16, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
initial = generate_single_state(10, [3, 3, 3, 3, 3, 0, 0, 0, 0, 0])

We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [7]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 250
trunc_tol = 1e-10
foldername_1 = f"/work_fast/ge49cag/code_datas/Krylov_H10"
generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

[1, 1, 3, 5, 5, 6, 4, 4, 3, 1, 1]
2


Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [ ]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)
H_reduced_1 = remain_only_tridiagonal_elements(H_reduced_1)

/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:193: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:106: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


In [ ]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_ground, mpo_ref)

In [ ]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 250
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/Krylov_H10_restart"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)



[1, 2, 4, 8, 16, 32, 64, 126, 214, 250, 250, 250, 220, 127, 64, 32, 16, 8, 4, 2, 1]
2
[1, 2, 4, 8, 16, 32, 64, 125, 211, 250, 250, 250, 211, 125, 64, 32, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 64, 125, 207, 250, 250, 250, 209, 125, 64, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 64, 126, 207, 250, 250, 250, 207, 125, 64, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 64, 126, 206, 250, 250, 250, 206, 125, 64, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 205, 125, 64, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 208, 126, 64, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 207, 126, 64, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 207, 126, 64, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 209, 127, 64, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 209, 127, 64, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 2

In [ ]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

In [ ]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_ground, mpo_ref)

restart:

In [ ]:
N_Krylov_3 = 30
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 250
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/Krylov_H10_restart_2"
generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)



[1, 2, 4, 8, 16, 32, 64, 127, 229, 250, 250, 250, 234, 128, 64, 32, 16, 8, 4, 2, 1]
2
[1, 2, 4, 8, 16, 32, 64, 127, 224, 250, 250, 250, 228, 128, 64, 32, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 64, 127, 224, 250, 250, 250, 226, 128, 64, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 64, 127, 224, 250, 250, 250, 228, 128, 64, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 64, 127, 227, 250, 250, 250, 227, 128, 64, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 64, 127, 227, 250, 250, 250, 227, 128, 64, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 227, 128, 64, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 2

In [ ]:

H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)
H_reduced_3 = remain_only_tridiagonal_elements(H_reduced_3)

In [ ]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_ground, mpo_ref)

ritz energy error: (0.0004171927154672517+0j)
ritz energy error: (0.0001335524906398433+0j)
ritz energy error: (0.00011517639941160951+0j)
ritz energy error: (0.00011464209390510405+0j)
ritz energy error: (0.0001139721012783923+0j)


restart:

In [ ]:
N_Krylov_4 = 30
psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 250
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/Krylov_H10_3"
generate_krylov_space_in_disk(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4)


In [ ]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)
H_reduced_4 = remain_only_tridiagonal_elements(H_reduced_4)

In [ ]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_ground, mpo_ref)

In [ ]:
e_final, v_final = np.linalg.norm(H_reduced_4)
print(e_final[0] - e_ground)